In [ ]:
import numpy as np
import pandas as pd

In [ ]:
m_path =  [] 
# input path(s) to the raw NanoDrop output .csv file(s) here
rawdata = []
# leave blank
for path in m_path:
    df1 = pd.read_csv(path,encoding='utf-16', delimiter='\t',usecols=['Nucleic Acid(ng/uL)'])
    df1['Consolidated Nano Raw'] = path
    rawdata.append(df1)
nano_rawdata = pd.concat(rawdata, ignore_index=True)
nano_rawlist = nano_rawdata['Nucleic Acid(ng/uL)'].tolist()
print(nano_rawlist)

In [ ]:
nanoen = list(enumerate(nano_rawlist))
print(nanoen)

In [ ]:
source = input('What is the source of these readings? (type either SP or ST):')
#inputs should only equal 'SP' or 'ST'

In [ ]:
nanoavg = []
# leave blank
index_map = {spot:round(nano,1) for spot, nano in nanoen}
skip_until = 0
cutoff = ((0,99,2),(99,299,5),(299,10000,10))
cutval = []
for spot, nano in nanoen:
    if spot < skip_until:
        continue
    if source == "SP" and index_map[spot] < 35:
        continue
    if source == "SP" and index_map[spot] > 95:
        continue
    for n1, n2, cut in cutoff:
        if (index_map[spot] > n1) and (index_map[spot] < n2 or index_map[spot] == n2):
            cutval = cut
            break
        else:
            continue
    if int(spot + 1) in index_map.keys():
        if abs(index_map[spot] - index_map[spot+1]) <= cutval:
            nanoavg.append((round((index_map[spot]+index_map[spot+1])/2,1)))
            skip_until = spot + 2
        else:
            combos = [(0,1),(0,2),(1,2),(0,3),(1,3),(2,3),
                      (0,4),(1,4),(2,4),(3,4),(0,5),(1,5),
                      (2,5),(3,5),(4,5),(0,6),(1,6),(2,6),
                      (3,6),(4,6),(5,6),(0,7),(1,7),(2,7),
                      (3,7),(4,7),(5,7),(6,7),(0,8),(1,8),
                      (2,8),(3,8),(4,8),(5,8),(6,8),(7,8)]
            for spot1,spot2 in combos:
                if source == "SP" and (index_map[spot + spot1] < 35 or index_map[spot + spot2] < 35 
                                       or index_map[spot + spot1] > 95 or index_map[spot +spot2] > 95):
                    continue
                if (spot + spot1) and (spot + spot2) in index_map.keys():
                    if (abs(index_map[spot + spot1]-index_map[spot + spot2])) <= cutval:
                        nanoavg.append((round((index_map[spot + spot1]+index_map[spot + spot2])/2,1)))
                        if spot1 > spot2:
                            skip_until = spot + spot1 + 1
                            break
                        elif spot2 > spot1:
                            skip_until = spot + spot2 + 1
                            break
print(nanoavg)

In [ ]:
code_path = 
# input path to .csv file for the list of codes exported by the NanoDrop
pcodes= pd.read_csv(code_path, delimiter='\t',usecols=['codes'])

codes1 = np.array([pcodes])
pcodes['Nanodrop Average [ng/uL]'] = nanoavg
print(pcodes)

In [ ]:
df2 = pd.DataFrame(pcodes)
df2.to_csv()
# input the path to where the output file should go including the name of the output file